In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from rasterio.transform import from_origin
import warnings
from datetime import datetime
import geopandas as gpd
from shapely.geometry import Point
from geopandas.tools import sjoin

# Enable interactive mode for matplotlib (if needed)
%matplotlib qt

# Suppress specific warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Function to perform IDW interpolation using cKDTree
def idw_interpolation(points, values, xi, yi, power=2, k=10):
    tree = cKDTree(points)
    distances, indices = tree.query(np.c_[xi.ravel(), yi.ravel()], k=k)
    weights = 1.0 / (distances ** power)
    weights[distances == 0] = np.inf  # Handle division by zero at data points
    z_interp = np.sum(weights * values[indices], axis=1) / np.sum(weights, axis=1)
    z_interp[~np.isfinite(z_interp)] = np.nan  # Handle areas with no data
    return z_interp.reshape(xi.shape)

# Load the CSV file
file_path = r"F:\DataExtractionWaterLevel\AllForcing_2012\SandyCS_0.021_Ocean_0.20_AllForcing_29082012_190000_mesh2d_s1.csv"
df = pd.read_csv(file_path)

# Extract latitude, longitude, and wind speed from the CSV
lat = df['Y'].values
lon = df['X'].values
data_value = df['Z'].values

# Extract timestamp from the filename (assuming the filename contains the date-time string as mentioned)
file_name = os.path.basename(file_path)
date_str = file_name.split('_')[-4] + '_' + file_name.split('_')[-3]
timestamp = datetime.strptime(date_str, "%d%m%Y_%H%M%S")

# Create a raster grid (defining the bounds from the data itself)
xmin, xmax = lon.min(), lon.max()
ymin, ymax = lat.min(), lat.max()
res = 0.05  # Define resolution (~1 km, adjust as needed)
x_grid = np.arange(xmin, xmax, res)
y_grid = np.arange(ymin, ymax, res)
x_mesh, y_mesh = np.meshgrid(x_grid, y_grid)

# IDW interpolation
points = np.column_stack((lon, lat))
grid_z = idw_interpolation(points, data_value, x_mesh, y_mesh, power=2, k=10)

# Clip the interpolated values to be within -5 to 5
grid_z = np.clip(grid_z, -5, 5)

# Load the shapefile (adjust the file path to your shapefile location)
shapefile_path = r"G:\My Drive\Deflt3D FM Codes - Vtech\Result_Map\shp_shoreline\SRTM10mShorelineMaskerUse.shp"
gdf = gpd.read_file(shapefile_path)

# Create a GeoDataFrame for the interpolated data points
interpolated_points = [Point(x, y) for x, y in zip(x_mesh.ravel(), y_mesh.ravel())]
interpolated_gdf = gpd.GeoDataFrame({'Water Level': grid_z.ravel()}, geometry=interpolated_points, crs=gdf.crs)

# Perform a spatial join to keep only the points inside the shapefile
interpolated_within_shape = sjoin(interpolated_gdf, gdf, how="inner", predicate="intersects")


# Create a masked grid where points outside the shapefile are NaN
grid_z_masked = np.full_like(grid_z, np.nan)
for point, value in zip(interpolated_within_shape.geometry, interpolated_within_shape['Water Level']):
    # Find the nearest grid cell for each point
    idx_x = np.argmin(np.abs(x_grid - point.x))
    idx_y = np.argmin(np.abs(y_grid - point.y))
    grid_z_masked[idx_y, idx_x] = value

# Plot the masked wind speed map (values outside the shapefile will be NaN)
plt.figure(figsize=(10, 8))
contour = plt.contourf(x_mesh, y_mesh, grid_z_masked, cmap='seismic', levels=100, vmin=-2, vmax=2)
cbar = plt.colorbar(contour)
cbar.set_label('Water Level (m)')

# Plot the shapefile boundary (optional, to visualize the area)
gdf.boundary.plot(ax=plt.gca(), color='black')

plt.title(f'Water Level Map on {timestamp.strftime("%Y-%m-%d %H:%M:%S")}')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Fill everything outside the shapefile with white color
plt.gca().set_facecolor('white')

plt.show()

